# Advent of Code challenge 2021

## Day 5: Hydrothermal Venture

### Part 1 - Find Overlapping Points in a Grid

Good references I found helpful:


In [1]:
import numpy as np
from typing import NamedTuple

In [2]:
test_input = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
"""

Working through how to parse instructions

In [76]:
lines = test_input.strip().split('\n')
lines[:3]

['0,9 -> 5,9', '8,0 -> 0,8', '9,4 -> 3,4']

In [34]:
point_list = [sum([part.split(',') for part in line.split(' -> ')], []) for line in lines]
point_list[:3]

[['0', '9', '5', '9'], ['8', '0', '0', '8'], ['9', '4', '3', '4']]

Get max X and Y points to get shape of grid tracker

In [38]:
x_pts = sum([[x0, x1] for x0, y0, x1, y1 in point_list], [])
max_x = max(sum([[int(x0), int(x1)] for x0, y0, x1, y1 in point_list], []))
max_x

9

In [3]:
class Point(NamedTuple):
    y0: int
    y1: int
    x0: int
    x1: int

Now each list element is a Point tuple containing the labeled beg and end points for X and Y coordinates

In [79]:
point_list = [sum([part.split(',') for part in line.split(' -> ')], []) for line in lines]
int_point_list = [[int(y0), int(y1), int(x0), int(x1)] for x0, y0, x1, y1 in point_list]
hori_vert_lines = [[y0, y1, x0, x1] for y0, y1, x0, x1 in int_point_list if (x0 == x1) | (y0 == y1)]
hori_vert_lines

[[9, 9, 0, 5],
 [4, 4, 9, 3],
 [2, 1, 2, 2],
 [0, 4, 7, 7],
 [9, 9, 0, 2],
 [4, 4, 3, 1]]

### Make a grid of 0s and process each instruction to increment each element accordingly

In [20]:
class Hydro:
    def __init__(self, puz_input):
        self.puz_input = puz_input
        self.max_x = 0
        self.max_y = 0
        self.vent_lines = self.get_vent_lines()
        self.grid = self.make_init_grid()
        
        
    def __repr__(self):
        info = [f"\nNum lines added: {len(self.vent_lines)}",
                f"Num overlap points: [ {self.num_overlap_pts} ]"]
        return '\n'.join(info)
    
            
    def get_vent_lines(self):
        """Extract vent lines (as a set of beg, end X,Y points) from puzzle input."""
        lines = self.puz_input.strip().split('\n')
        point_list = [sum([part.split(',') for part in line.split(' -> ')], []) for line in lines]
        int_point_list = [[int(y0), int(y1), int(x0), int(x1)] for x0, y0, x1, y1 in point_list]
        hori_vert_lines = [[y0, y1, x0, x1] for y0, y1, x0, x1 in int_point_list if (x0 == x1) | (y0 == y1)]
        
        # Get max X, Y values to set a matching grid array with shape (X, Y)
        self.max_x = max(sum([[x0, x1] for y0, y1, x0, x1 in hori_vert_lines], [])) + 1
        self.max_y = max(sum([[y0, y1] for y0, y1, x0, x1 in hori_vert_lines], [])) + 1
        
        return [Point(y0, y1, x0, x1) for y0, y1, x0, x1 in hori_vert_lines]
    
    
    def make_init_grid(self):
        """Make a grid of 0s to track where vent lines are located."""
        return np.array([[0] * self.max_x] * self.max_y)
    
    
    def add_vent_lines_to_grid(self):
        """Loop over vent lines and add 'points' to grid to create vent lines.
            - Min, max are used to ensure the smaller point comes before the larger point when
              accessing the numpy array
        """
        for line in self.vent_lines:
            if line.y0 == line.y1:
                self.grid[line.y0, min(line.x0, line.x1):max(line.x0, line.x1) + 1] += 1
            elif line.x0 == line.x1:
                self.grid[min(line.y0, line.y1):max(line.y0, line.y1) + 1, line.x0] += 1
    
    
    def find_num_overlap_points(self):
        """Find the number of points where at least 2 lines overlap."""
        self.num_overlap_pts = len(self.grid[self.grid >= 2])
        
        
    def solve(self):
        """Main function to solve hydrothermal vent puzzle."""
        self.add_vent_lines_to_grid()
        self.find_num_overlap_points()

In [118]:
hydro_test = Hydro(puz_input=test_input)
display(hydro_test.vent_lines)

[Point(y0=9, y1=9, x0=0, x1=5),
 Point(y0=4, y1=4, x0=9, x1=3),
 Point(y0=2, y1=1, x0=2, x1=2),
 Point(y0=0, y1=4, x0=7, x1=7),
 Point(y0=9, y1=9, x0=0, x1=2),
 Point(y0=4, y1=4, x0=3, x1=1)]

In [109]:
# Test adding a line of points from 0,9 -> 5,9
# numpy accessors are like this: [Y is rows, X is cols]
test_grid = hydro_test.grid
test_grid[9, 0:5+1] += 1
test_grid

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])

In [110]:
# Test adding a line of points from 7,0 -> 7,4
# numpy accessors are like this: [Y is rows, X is cols]
test_grid[0:4+1, 7] += 1
test_grid

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])

Test adding points to grid as a class function

In [119]:
hydro_test.add_vent_lines_to_grid()
display(hydro_test.grid)

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 1, 2, 1, 1, 1, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 2, 2, 1, 1, 1, 0, 0, 0, 0]])

Find number of points where at least 2 lines overlap (grid value >= 2)

In [127]:
points_gte_2 = hydro_test.grid[hydro_test.grid >= 2]
print(points_gte_2)
len(points_gte_2)

[2 2 2 2 2]


5

In [21]:
hydro_test = Hydro(puz_input=test_input)
hydro_test.solve()
print(hydro_test)


Num lines added: 6
Num overlap points: [ 5 ]


In [22]:
# Run with final puzzle input
with open('adv_2021_d5_input.txt', 'r') as f:
    hydro_pt1 = Hydro(puz_input=f.read())
    hydro_pt1.solve()
    print(hydro_pt1)


Num lines added: 321
Num overlap points: [ 4655 ]


### Part 2 - Same as Pt.1 but with Diagonal Lines!

Plan:
- Modify `get_vent_lines` to no longer remove diagonal lines
- Modify `add_vent_lines_to_grid` function to handle diagonal lines

In [31]:
class Hydro2:
    def __init__(self, puz_input):
        self.puz_input = puz_input
        self.max_x = 0
        self.max_y = 0
        self.vent_lines = self.get_vent_lines()
        self.grid = self.make_init_grid()
        
        
    def __repr__(self):
        info = [f"\nLines added: {len(self.vent_lines)}",
                f"Overlapping points: [ {self.num_overlap_pts} ]"]
        return '\n'.join(info)
    
            
    def get_vent_lines(self):
        """Extract vent lines (as a set of beg, end X,Y points) from puzzle input."""
        lines = self.puz_input.strip().split('\n')
        point_list = [sum([part.split(',') for part in line.split(' -> ')], []) for line in lines]
        int_point_list = [[int(y0), int(y1), int(x0), int(x1)] for x0, y0, x1, y1 in point_list]
        
        # Get max X, Y values to set a matching grid array with shape (X, Y)
        self.max_x = max(sum([[x0, x1] for y0, y1, x0, x1 in int_point_list], [])) + 1
        self.max_y = max(sum([[y0, y1] for y0, y1, x0, x1 in int_point_list], [])) + 1
        
        return [Point(y0, y1, x0, x1) for y0, y1, x0, x1 in int_point_list]
    
    
    def make_init_grid(self):
        """Make a grid of 0s to track where vent lines are located."""
        return np.array([[0] * self.max_x] * self.max_y)
    
    
    def add_vent_lines_to_grid(self):
        """Loop over vent lines and add 'points' to grid to create vent lines.
            - Min, max are used to ensure the smaller point comes before the larger point when
              accessing the numpy array
        """
        for line in self.vent_lines:
            # Horizontal
            if line.y0 == line.y1:
                self.grid[line.y0, min(line.x0, line.x1):max(line.x0, line.x1) + 1] += 1
            
            # Vertical
            elif line.x0 == line.x1:
                self.grid[min(line.y0, line.y1):max(line.y0, line.y1) + 1, line.x0] += 1
            
            # Diagonal
            else:
                x_min, x_max = [min(line.x0, line.x1), max(line.x0, line.x1)]
                y_min, y_max = [min(line.y0, line.y1), max(line.y0, line.y1)]
                
                x_flip = 1 if line.x0 < line.x1 else -1
                y_flip = 1 if line.y0 < line.y1 else -1
                
                x_diag = [x for x in range(x_min, x_max + 1)][::x_flip]
                y_diag = [y for y in range(y_min, y_max + 1)][::y_flip]
                
#                 print(f"({line.x0}, {line.y0} -> {line.x1}, {line.y1})")
#                 print((x_diag, y_diag))
                self.grid[(y_diag, x_diag)] += 1
    
    
    def find_num_overlap_points(self):
        """Find the number of points where at least 2 lines overlap."""
        self.num_overlap_pts = len(self.grid[self.grid >= 2])
        
        
    def solve(self):
        """Main function to solve hydrothermal vent puzzle."""
        self.add_vent_lines_to_grid()
        self.find_num_overlap_points()

In [9]:
hydro_test2 = Hydro2(puz_input=test_input)
display(hydro_test2.vent_lines)

[Point(y0=9, y1=9, x0=0, x1=5),
 Point(y0=0, y1=8, x0=8, x1=0),
 Point(y0=4, y1=4, x0=9, x1=3),
 Point(y0=2, y1=1, x0=2, x1=2),
 Point(y0=0, y1=4, x0=7, x1=7),
 Point(y0=4, y1=0, x0=6, x1=2),
 Point(y0=9, y1=9, x0=0, x1=2),
 Point(y0=4, y1=4, x0=3, x1=1),
 Point(y0=0, y1=8, x0=0, x1=8),
 Point(y0=5, y1=2, x0=5, x1=8)]

Test diagonal line case in `add_vent_lines_to_grid` function

In [10]:
lines2 = hydro_test2.vent_lines
for line in lines2:
    if line.y0 == line.y1:
        pass
    elif line.x0 == line.x1:
        pass
    else: # Diagonal
        x_min, x_max = [min(line.x0, line.x1), max(line.x0, line.x1)]
        y_min, y_max = [min(line.y0, line.y1), max(line.y0, line.y1)]

        x_flip = 1 if line.x0 < line.x1 else -1
        y_flip = 1 if line.y0 < line.y1 else -1

        x_diag = [x for x in range(x_min, x_max + 1)][::x_flip]
        y_diag = [y for y in range(y_min, y_max + 1)][::y_flip]

        print(f"({line.x0}, {line.y0} -> {line.x1}, {line.y1})")
        print((x_diag, y_diag))
#         self.grid[(x_diag, y_diag)] += 1

(8, 0 -> 0, 8)
([8, 7, 6, 5, 4, 3, 2, 1, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8])
(6, 4 -> 2, 0)
([6, 5, 4, 3, 2], [4, 3, 2, 1, 0])
(0, 0 -> 8, 8)
([0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8])
(5, 5 -> 8, 2)
([5, 6, 7, 8], [5, 4, 3, 2])


In [179]:
hydro_test.grid

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

I made a diagonal line, now we can solve the whole test puzzle and then the final puzzle

In [28]:
hydro_test2 = Hydro2(puz_input=test_input)
hydro_test2.solve()
print(hydro_test2)

(8, 0 -> 0, 8)
([8, 7, 6, 5, 4, 3, 2, 1, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8])
(6, 4 -> 2, 0)
([6, 5, 4, 3, 2], [4, 3, 2, 1, 0])
(0, 0 -> 8, 8)
([0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8])
(5, 5 -> 8, 2)
([5, 6, 7, 8], [5, 4, 3, 2])

Lines added: 10
Overlapping points: [ 12 ]


In [33]:
# Run with final puzzle input
with open('adv_2021_d5_input.txt', 'r') as f:
    hydro_pt2 = Hydro2(puz_input=f.read())
    hydro_pt2.solve()
    print(hydro_pt2)


Lines added: 500
Overlapping points: [ 20500 ]
